#### This file can be used to reproduce the Yosemite related part in Table 2, 3, 7 and Table 5, 6 in the paper.
This file reads data directly from Github. If you would like to use the Trip Construction file to generate your own output in Binder, simply change the *pmatrix_url* in Chunk[5] to the one under option 2 and run the chunks below.

In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import re
import string
import scipy
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [3]:
def split_date(df):
    df['datetaken'] = pd.to_datetime(df['datetaken'])
    df['date'] = [d.date() for d in df['datetaken']]
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    return df

def subset_data(input,month):
    subset = input[input['month'] == month]
    return subset

In [4]:
park = 'yosemite'

In [5]:
path = "https://raw.githubusercontent.com/meilinshi/Socially-aware-Huff-model/main/Data/"
input_url = path+park+"_NP_cluster.csv"
position_url = path+park+"_NP_coords.csv"
dist_matrix_url = path+park+"_NP_dist_matrix.csv"

# load the Flickr photo with cluster information
df = pd.read_csv(input_url)
df = split_date(df) 

# load the position of attractions
position = pd.read_csv(position_url) 
position['coord'] = list(zip(position.Latitude, position.Longitude))
places = position['Clusters from Data'].values

# load the distance matrix
dist_matrix = pd.read_csv(dist_matrix_url,index_col=0)


# load probability matrix for the observed trips in the park
# option 1: read data directly from Github
pmatrix_url = path+park+"_pmatrix/"+park+"_NP_cluster_prob_matrix_"

# option 2: read data from Binder when you generate your own output from the Trip Construction file
#pmatrix_url = park+"_NP_cluster_prob_matrix_"

pmatrix_all = [pmatrix_url+str(i)+".csv" for i in range(1,13)]
pmatrix_summer = [pmatrix_url+str(i)+".csv" for i in range(5,10)]
pmatrix_non_summer = [pmatrix_url+str(i)+".csv" for i in range(1,5)]+[pmatrix_url+str(i)+".csv" for i in range(10,13)]

### Construct distance matrix and attractiveness matrix needed for the model

In [6]:
# Calculate travel distance (in km) using google map distance matrix api
# import googlemaps
# API_key = 'xxxxx'
# gmaps = googlemaps.Client(key=API_key)

def get_dist_matrix(df):
    destinations = df.coord
    names = df['Clusters from Data'].values    
    dim = len(destinations)
    dist_matrix = np.zeros((dim, dim), float)        
    for i in range(dim):
        actual_distance = []
        origin = destinations[i]        
        for destination in destinations:
            result = gmaps.distance_matrix(origin, destination, mode='driving')['rows'][0]['elements'][0]['distance']['value']
            result = result/1000
            actual_distance.append(result)
        dist_matrix[i] = actual_distance       
    res = pd.DataFrame(data=dist_matrix, index = names, columns=names)
    return res

# generate a distance matrix by distance matrix API
# dist_matrix = get_dist_matrix(position)
# Here a download version of distance matrix in the data folder is used.

# generate attractiveness matrix, val indicates different measurements of attractiveness
def attr_matrix(df, month, val):
    attr_matrix = pd.DataFrame()
    df = subset_data(df, month)    
    attr_matrix['Places'] = position['Clusters from Data'].values
    attr_matrix['photo_views'] = df.groupby(['Cluster'])['views'].agg('sum')
    attr_matrix['num_uploaders'] = df.groupby(['Cluster'])['owner'].nunique()
    attr_matrix['num_of_photos'] = df.groupby(['Cluster']).size()
    attr_matrix['avg_view_per_user'] = attr_matrix['photo_views']/attr_matrix['num_uploaders']
    if val == 1:
        attr_matrix['total_attr'] = attr_matrix['num_of_photos'] #Aj1
    if val == 2:
        attr_matrix['total_attr'] = attr_matrix['num_uploaders'] #Aj2
    if val == 3:
        attr_matrix['total_attr'] = attr_matrix['num_of_photos'] * attr_matrix['avg_view_per_user'] #Aj3  
    attr_matrix = attr_matrix.fillna(0)
    attr_matrix['total_attr_log'] = np.log(attr_matrix['total_attr']+1)
    attr_matrix = attr_matrix.set_index('Places')
    return attr_matrix

# generate attractiveness matrix with num_of_photos*avg_view_per_user, Aj3, without temporal factor
def attr_matrix_all(df):
    attr_matrix = pd.DataFrame() 
    attr_matrix['Places'] = position['Clusters from Data'].values
    attr_matrix['photo_views'] = df.groupby(['Cluster'])['views'].agg('sum')
    attr_matrix['num_uploaders'] = df.groupby(['Cluster'])['owner'].nunique()
    attr_matrix['num_of_photos'] = df.groupby(['Cluster']).size()
    attr_matrix['avg_view_per_user'] = attr_matrix['photo_views']/attr_matrix['num_uploaders']
    attr_matrix['total_attr'] = attr_matrix['num_of_photos'] * attr_matrix['avg_view_per_user']
    attr_matrix['total_attr_log'] = np.log(attr_matrix['total_attr'])
    attr_matrix = attr_matrix.set_index('Places')
    return attr_matrix


# to include the neighboring effect
# select K neighbors
def neighbors(dest, dist_matrix, K):
    destinations = dist_matrix.index.values
    dist_tp = np.transpose(dist_matrix)
    neighbors = dist_tp.nsmallest(10, [dest])[1:K+1].index.values   
    return neighbors


# calculate centrality score based on K neighbors, attraction matrix and distance matrix
def centrality(dest, attr_matrix, K):
    neighbor_lst = neighbors(dest, dist_matrix, K)
    c = 0
    dist = 0
    for p in neighbor_lst:
        c += attr_matrix.loc[p]['total_attr_log']/dist_matrix.loc[dest][p]
        dist += dist_matrix.loc[dest][p]
        c = c/dist
    return c

### Ordinary Least Squares (OLS) Calibration

In [7]:
def getComplement(item, lst):
    results = []
    for num in lst:
        if num != item: 
            results.append(num)
    return results

# OLS dependent variable
def read_actual(pmatrix, origin):
    num = 0
    denom = 0
    result = []
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)   
    actual_pmatrix = pd.read_csv(pmatrix, index_col=0)
    for i in range(len(dests)):
        num = actual_pmatrix.loc[origin].values[i]
        denom = np.mean(actual_pmatrix.loc[origin])
        result.append(num/denom)
    return result

# OLS independent variables
# attractiveness (including Social Influence), distance, centrality, without temporal factor
def log_transform_x_NT(origin,K):
    X1, X2, X3 = [],[],[]
    total_centrality = 0
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)
    attr_mat = attr_matrix_all(df)     
    for dest in dests:
        total_centrality += centrality(dest, attr_mat, K)
        X1.append(attr_mat.loc[dest]['total_attr_log']/np.mean(attr_mat['total_attr_log']))
        X2.append(dist_matrix.loc[origin][dest]/ np.mean(dist_matrix.loc[origin]))
        X3.append(centrality(dest, attr_mat, K)/(total_centrality/len(dests)))
    var_table = pd.DataFrame()
    X1 = [x + 1 for x in X1]
    X3 = [x + 1 for x in X3]
    var_table['x1'] = np.nan_to_num(np.log(X1))
    var_table['x2'] = np.nan_to_num(np.log(X2))
    var_table['x3'] = np.nan_to_num(np.log(X3))
    return var_table

# OLS independent variables
# val indicates different measurements of attractiveness
def log_transform_x(origin,K,month,val):
    X1, X2, X3 = [],[],[]
    total_centrality = 0
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)    
    attr_mat = attr_matrix(df, month, val)
    for dest in dests:
        total_centrality += centrality(dest, attr_mat, K)
        X1.append(attr_mat.loc[dest]['total_attr_log']/np.mean(attr_mat['total_attr_log']))
        X2.append(dist_matrix.loc[origin][dest]/ np.mean(dist_matrix.loc[origin]))
        X3.append(centrality(dest, attr_mat, K)/(total_centrality/len(dests)))
    var_table = pd.DataFrame()
    X1 = [x + 1 for x in X1]
    X3 = [x + 1 for x in X3]
    var_table['x1'] = np.nan_to_num(np.log(X1))
    var_table['x2'] = np.nan_to_num(np.log(X2))
    var_table['x3'] = np.nan_to_num(np.log(X3))
    return var_table

#### Reproduce Table 2 in the paper

In [8]:
# Y value used for both table 2 and 3
Y_res = []
for place in places:
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res = np.append(Y_res, np.round(log_Y,10))

In [9]:
def var_tbl(val):
    var_table = []
    for place in places:
        for i in range(1,13):
            tbl = log_transform_x(place,2,i,val)
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def clear_var_tbl(df):
    df['Y'] = Y_res
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl2_results(df):
    df = clear_var_tbl(df)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    r2 = round(model.rsquared,3)
    aic = round(model.aic,1)
    return [r2,aic]

In [10]:
tbl2_results=[]
for val in range(1,4):
    df_tbl2 = var_tbl(val)
    tbl2_results.append(return_tbl2_results(df_tbl2))
    
def generate_tbl2(tbl2_results):
    df_tbl2 = pd.DataFrame(columns=['Aj1','Aj2','Aj3'])
    df_tbl2['Aj1'] = tbl2_results[0]
    df_tbl2['Aj2'] = tbl2_results[1]
    df_tbl2['Aj3'] = tbl2_results[2]
    df_tbl2 = df_tbl2.T
    df_tbl2.columns =['R2', 'AIC']
    df_tbl2['delta_AIC'] = df_tbl2['AIC'] - min(df_tbl2['AIC'])
    df_tbl2['w_i'] = np.exp(-0.5*df_tbl2['delta_AIC'])/sum(np.exp(-0.5*df_tbl2['delta_AIC']))
    return df_tbl2

park_tbl2 = generate_tbl2(tbl2_results)
park_tbl2

,R2,AIC,delta_AIC,w_i
Aj1,0.715,2401.7,25.4,0.000003
Aj2,0.717,2393.0,16.7,0.000236
Aj3,0.721,2376.3,0.0,0.999761


#### Reproduce Table 3 in the paper

In [11]:
def var_tbl3(val, time):
    var_table = []
    for place in places:
        for i in range(1,13):
            if time == True:
                tbl = log_transform_x(place,2,i,val)
            else:
                tbl = log_transform_x_NT(place,2)
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def return_tbl3_results(df, neighbor):
    df = clear_var_tbl(df)
    if neighbor == True:
        X = df[['x1', 'x2','x3']]
    else:
        X = df[['x1', 'x2']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    r2 = round(model.rsquared,3)
    aic = round(model.aic,1)
    return [r2,aic]

In [12]:
def model_selection(neighbor, time):
    df_tbl3 = var_tbl3(3,time) #Aj3 is used here
    return return_tbl3_results(df_tbl3, neighbor)
    
def generate_tbl3():
    df_tbl3 = pd.DataFrame(columns=['SA_model','SA_model_no_N','SA_model_no_T','Huff_model'])
    df_tbl3['SA_model'] = model_selection(neighbor=True, time=True)
    df_tbl3['SA_model_no_N'] = model_selection(neighbor=False, time=True)
    df_tbl3['SA_model_no_T'] = model_selection(neighbor=True, time=False)
    df_tbl3['Huff_model'] = model_selection(neighbor=False, time=False)
    df_tbl3 = df_tbl3.T
    df_tbl3.columns =['R2', 'AIC']
    df_tbl3['delta_AIC'] = df_tbl3['AIC'] - min(df_tbl3['AIC'])
    df_tbl3['w_i'] = np.exp(-0.5*df_tbl3['delta_AIC'])/sum(np.exp(-0.5*df_tbl3['delta_AIC']))
    return df_tbl3

park_tbl3 = generate_tbl3()
park_tbl3

,R2,AIC,delta_AIC,w_i
SA_model,0.721,2376.3,1.3,3.429895e-01
SA_model_no_N,0.721,2375.0,0.0,6.570105e-01
SA_model_no_T,0.714,2412.4,37.4,4.968960e-09
Huff_model,0.714,2410.6,35.6,1.222167e-08


#### Reproduce Table 5 in the paper

In [13]:
def Y_res_place(place):
    Y_res = []
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res = np.append(Y_res, np.round(log_Y,10))
    return Y_res

def var_table_place(place):
    var_table = []
    for i in range(1,13):
        tbl = log_transform_x(place,2,i,3) #Aj3
        var_table.append(tbl)
    return pd.concat(var_table)

def clear_var_tbl_place(df, place):
    df['Y'] = Y_res_place(place)
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl_4_5_results(df,place):
    df = clear_var_tbl_place(df,place)
    num_obs = len(df)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']    
    model = sm.OLS(Y,X).fit()    
    alpha = round(model.params[0],4)
    alpha_p = round(model.pvalues[0],3)
    beta = round(model.params[1],4)
    beta_p = round(model.pvalues[1],3)
    theta = round(model.params[2],4)
    theta_p = round(model.pvalues[2],3)
    mse = round(model.mse_resid,3)
    r2 = round(model.rsquared,3)    
    return [place, num_obs, alpha, alpha_p, beta, beta_p, theta, theta_p, mse, r2]

def mark_significance(lst):
    p_val = []
    for item in lst:
        if item <= 0.001:
            p_val.append('***')
        elif item > 0.001 and item <= 0.01:
            p_val.append('**')
        elif item > 0.01 and item <= 0.05:
            p_val.append('*')
        else:
            p_val.append(' ')
    return p_val

In [14]:
tbl_4_5_results = []
for place in places:
    var_table = var_table_place(place)
    tbl_4_5_results.append(return_tbl_4_5_results(var_table,place))

def generate_tbl_4_5(tbl_results):
    df_tbl_4_5 = pd.DataFrame(data=tbl_results,columns=['Place','num_obs','alpha','alpha_p','beta','beta_p','theta','theta_p','MSE','R2'])
    df_tbl_4_5 = df_tbl_4_5[df_tbl_4_5['num_obs'] >= 30]
    df_tbl_4_5['alpha_sig'] = mark_significance(df_tbl_4_5['alpha_p'])
    df_tbl_4_5['beta_sig'] = mark_significance(df_tbl_4_5['beta_p'])
    df_tbl_4_5['theta_sig'] = mark_significance(df_tbl_4_5['theta_p'])
    df_tbl_4_5 = df_tbl_4_5[['Place','alpha','alpha_sig','beta','beta_sig','theta','theta_sig','MSE','R2']]
    return df_tbl_4_5

generate_tbl_4_5(tbl_4_5_results)

,Place,alpha,alpha_sig,beta,beta_sig,theta,theta_sig,MSE,R2
0,Mariposa Grove of Giant Sequoias,1.6864,***,-0.1023,,-0.1060,,0.433,0.743
1,Tioga Lake,1.4482,*,0.0467,,0.0381,,0.770,0.615
2,Tuolumne Grove,0.7325,*,-1.1656,**,0.4270,***,0.368,0.850
3,Tuolumne Meadows,0.8987,**,-0.4552,***,0.0985,,0.388,0.776
5,Olmsted Point,0.2791,,-0.0827,,0.3661,**,0.399,0.731
6,Tenaya Lake,0.9528,*,-0.3699,***,0.0838,,0.495,0.786
7,Wildcat Falls,1.6585,***,-0.0451,,-0.0818,,0.355,0.799
8,Mirror Lake,1.8888,***,-0.0490,,-0.2065,,0.343,0.802
9,Vernal Falls,0.9077,***,-0.0532,,0.0236,,0.294,0.666
10,El Capitan Meadow,1.1824,***,-0.1147,,0.0205,,0.208,0.827


#### Reproduce Table 6 in the paper

In [15]:
def Y_res_place(place,summer):
    Y_res = []
    if summer == True:
        for file in pmatrix_summer:
            Y = read_actual(file, place)
            log_Y = np.nan_to_num(np.log(Y))
            Y_res = np.append(Y_res, np.round(log_Y,10))
    else:
        for file in pmatrix_non_summer:
            Y = read_actual(file, place)
            log_Y = np.nan_to_num(np.log(Y))
            Y_res = np.append(Y_res, np.round(log_Y,10))
    return Y_res

def var_table_place(place,summer):
    var_table = []
    if summer == True:
        for i in range(5,10):
            tbl = log_transform_x(place,2,i,3) #Aj3
            var_table.append(tbl)
    else: 
        for i in [1,2,3,4,10,11,12]:
            tbl = log_transform_x(place,2,i,3) #Aj3
            var_table.append(tbl)
    return pd.concat(var_table)

def clear_var_tbl_place(df,place,summer):
    df['Y'] = Y_res_place(place,summer)
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl_6_results(df,place,summer):
    df = clear_var_tbl_place(df,place,summer)
    num_obs = len(df)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']    
    model = sm.OLS(Y,X).fit()    
    alpha = round(model.params[0],4)
    alpha_p = round(model.pvalues[0],3)
    beta = round(model.params[1],4)
    beta_p = round(model.pvalues[1],3)
    theta = round(model.params[2],4)
    theta_p = round(model.pvalues[2],3)
    r2 = round(model.rsquared,3)    
    return [place, num_obs, alpha, alpha_p, beta, beta_p, theta, theta_p, r2]

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [16]:
tbl_6_results_summer = []
for place in places:
    var_table = var_table_place(place,summer=True)
    tbl_6_results_summer.append(return_tbl_6_results(var_table,place,summer=True))

tbl_6_results_non_summer = []
for place in places:
    var_table = var_table_place(place,summer=False)
    tbl_6_results_non_summer.append(return_tbl_6_results(var_table,place,summer=False))

In [17]:
def clear_df(results, input_str):
        df = pd.DataFrame(data=results, columns = ['Place','num_obs','alpha','alpha_p','beta','beta_p','theta','theta_p','R2'])
        df['Time'] = input_str
        df = df[df['num_obs'] >= 30]
        return df

df_summer = clear_df(tbl_6_results_summer, "Summer")
df_non_summer = clear_df(tbl_6_results_non_summer, "Non-Summer")

sig_place = intersection(df_summer['Place'].values,df_non_summer['Place'].values)

def generate_tbl6(df1,df2):
    tbl_6 = pd.concat([df1, df2])
    tbl_6 = tbl_6[tbl_6['Place'].isin(sig_place)]
    tbl_6['alpha_sig'] = mark_significance(tbl_6['alpha_p'])
    tbl_6['beta_sig'] = mark_significance(tbl_6['beta_p'])
    tbl_6['theta_sig'] = mark_significance(tbl_6['theta_p'])
    tbl_6 = tbl_6 [['Place','Time','alpha','alpha_sig','beta','beta_sig','theta','theta_sig','R2']]
    tbl_6 = tbl_6.set_index(['Place','Time'])
    tbl_6 = tbl_6.sort_index(ascending=False)
    return tbl_6

generate_tbl6(df_summer,df_non_summer)

alpha alpha_sig    beta beta_sig   theta  \
Place                 Time                                                    
Wildcat Falls         Summer      1.7046         *  0.0519          -0.1768   
                      Non-Summer  1.4516         * -0.1670           0.0945   
Vernal Falls          Summer      0.8962        ** -0.1116        * -0.0355   
                      Non-Summer  0.9788       ***  0.0379           0.0978   
Valley Visitor Center Summer      0.8253       *** -0.0782          -0.0404   
                      Non-Summer  1.1169       *** -0.0841          -0.0643   
Tunnel View           Summer      0.4185           -0.4669        *  0.1651   
                      Non-Summer  0.7634           -0.0111           0.1197   
Sentinel Bridge       Summer      0.7097         * -0.2088       ** -0.0307   
                      Non-Summer  1.4738       *** -0.0374          -0.1927   
Lower Yosemite Fall   Summer      1.1861       ***  0.0315           0.0102   
                      Non-Summer  1.2295       *** -0.0319          -0.0319   
El Capitan Meadow     Summer      1.2480       *** -0.0869           0.0372   
                      Non-Summer  1.0437        ** -0.2023           0.0010   
Curry Village         Summer      1.3597       *** -0.0117          -0.0348   
                      Non-Summer  1.2933       *** -0.2013       ** -0.1963   
Bridalveil Falls      Summer      1.4542       ***  0.1098          -0.1979   
                      Non-Summer  0.9029           -0.1969          -0.0592   

                                 theta_sig     R2  
Place                 Time                         
Wildcat Falls         Summer                0.744  
                      Non-Summer            0.859  
Vernal Falls          Summer                0.679  
                      Non-Summer            0.693  
Valley Visitor Center Summer                0.705  
                      Non-Summer            0.784  
Tunnel View           Summer                0.756  
                      Non-Summer            0.607  
Sentinel Bridge       Summer                0.779  
                      Non-Summer        **  0.875  
Lower Yosemite Fall   Summer                0.841  
                      Non-Summer            0.787  
El Capitan Meadow     Summer                0.918  
                      Non-Summer            0.768  
Curry Village         Summer                0.800  
                      Non-Summer         *  0.800  
Bridalveil Falls      Summer                0.729  
                      Non-Summer            0.755

#### Reproduce Table 7 in the paper

In [18]:
def var_tbl_7(K, time):
    var_table = []
    if time == 0: #all-time
        month_range = range(1,13)
    if time == 1: # summer
        month_range = range(5,10)
    if time == 2: #non-summer
        month_range = [1,2,3,4,10,11,12]
    for place in places:
        for i in month_range:
            tbl = log_transform_x(place,K,i,3) #Aj3
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def clear_var_tbl(df, Y_val):
    df['Y'] = Y_val
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl7_results(df,Y_val):
    df = clear_var_tbl(df,Y_val)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    mse = round(model.mse_resid,6)
    r2 = round(model.rsquared,3)
    return [mse,r2]

In [19]:
Y_res_all,Y_res_summer,Y_res_non_summer = [],[],[]
for place in places:
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_all = np.append(Y_res_all, np.round(log_Y,10))
    for file in pmatrix_summer:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_summer = np.append(Y_res_summer, np.round(log_Y,10))
    for file in pmatrix_non_summer:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_non_summer = np.append(Y_res_non_summer, np.round(log_Y,10))

In [20]:
tbl7_all,tbl7_summer,tbl7_non_summer,tbl7_results=[],[],[],[]
for K in [2,3,5]:
    df_all = var_tbl_7(K,0)
    df_summer = var_tbl_7(K,1)
    df_non_summer = var_tbl_7(K,2)   
    tbl7_all.append(return_tbl7_results(df_all,Y_res_all))
    tbl7_summer.append(return_tbl7_results(df_summer,Y_res_summer))
    tbl7_non_summer.append(return_tbl7_results(df_non_summer,Y_res_non_summer))    
tbl7_results = tbl7_all+tbl7_summer+tbl7_non_summer

def generate_tbl7(results):
    df_tbl7 = pd.DataFrame(data=results)
    df_tbl7.columns =['MSE', 'R2']
    df_tbl7['Time'] = ["All_year"]*3+['Summer']*3+['Non_summer']*3
    df_tbl7['K'] = [2,3,5]*3
    df_tbl7 = df_tbl7.set_index(['Time', 'K'])    
    return df_tbl7

park_tbl7 = generate_tbl7(tbl7_results)
park_tbl7

MSE     R2
Time       K                 
All_year   2  0.373372  0.721
           3  0.373495  0.721
           5  0.373465  0.721
Summer     2  0.310437  0.714
           3  0.310878  0.714
           5  0.311528  0.713
Non_summer 2  0.430608  0.735
           3  0.430768  0.734
           5  0.431058  0.734